In [23]:
import pandas as pd

df_out_path = "/Users/joeyrich/Desktop/local/varseek/tests/pytest_output/test_varseek_ref/date_2025_03_07_time_1607_57/test_vk_ref/variants_updated_exploded_vk_info.csv"
df_ground_truth_path = "/Users/joeyrich/Desktop/local/varseek/tests/pytest_ground_truth/variants_updated_exploded_vk_info.csv"

In [24]:
df_out = pd.read_csv(df_out_path).fillna('NULL_FOR_TESTING')
df_ground_truth = pd.read_csv(df_ground_truth_path).fillna('NULL_FOR_TESTING')

/var/folders/wr/wdr46lxn7fbc_kty6kwbfhxc0000gn/T/ipykernel_8278/2847970535.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_out = pd.read_csv(df_out_path).fillna('NULL_FOR_TESTING')
/var/folders/wr/wdr46lxn7fbc_kty6kwbfhxc0000gn/T/ipykernel_8278/2847970535.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ground_truth = pd.read_csv(df_ground_truth_path).fillna('NULL_FOR_TESTING')


In [ ]:
df_out.columns

In [25]:
pd.Series.equals(df_out['vcrs_header'], df_ground_truth['vcrs_header'])

True

In [26]:
pd.Series.equals(df_out['VCRSs_with_overlapping_kmers'], df_ground_truth['VCRSs_with_overlapping_kmers'])

False

In [27]:
df_out.loc[df_out['VCRSs_with_overlapping_kmers'].ne(df_ground_truth['VCRSs_with_overlapping_kmers']), ['vcrs_header', 'VCRSs_with_overlapping_kmers']]

,vcrs_header,VCRSs_with_overlapping_kmers
4076,ENST00000241453:c.1897_1898insGGAAAGCGGGTGACCG...,"{'vcrs_05621', 'vcrs_05126', 'vcrs_05705', 'vc..."
4651,ENST00000241453:c.1883_1884insCTACCAGCCAGGTGAC...,"{'vcrs_05621', 'vcrs_05126', 'vcrs_05705', 'vc..."
5127,ENST00000241453:c.1881_1882insGTCCTCCTCAGATAAT...,"{'vcrs_05621', 'vcrs_05705', 'vcrs_04650', 'vc..."
5325,ENST00000241453:c.1877_1878insGGGTGACCGGCTCCTC...,"{'vcrs_05621', 'vcrs_05705', 'vcrs_04650', 'vc..."
5388,ENST00000241453:c.1877_1878insCCGTGACCGGCTCCTC...,"{'vcrs_05621', 'vcrs_05705', 'vcrs_04650', 'vc..."
5623,ENST00000241453:c.1815_1816insCCGGCTCCTCAGATAA...,"{'vcrs_05126', 'vcrs_05705', 'vcrs_04650', 'vc..."
5707,ENST00000241453:c.1903_1904insAGTCCACAGGTGACCG...,"{'vcrs_05621', 'vcrs_05126', 'vcrs_04552', 'vc..."


In [28]:
df_ground_truth.loc[df_out['VCRSs_with_overlapping_kmers'].ne(df_ground_truth['VCRSs_with_overlapping_kmers']), ['vcrs_header', 'VCRSs_with_overlapping_kmers']]

,vcrs_header,VCRSs_with_overlapping_kmers
4076,ENST00000241453:c.1897_1898insGGAAAGCGGGTGACCG...,"{'vcrs_04650', 'vcrs_05126', 'vcrs_05705', 'vc..."
4651,ENST00000241453:c.1883_1884insCTACCAGCCAGGTGAC...,"{'vcrs_05126', 'vcrs_04076', 'vcrs_05705', 'vc..."
5127,ENST00000241453:c.1881_1882insGTCCTCCTCAGATAAT...,"{'vcrs_04650', 'vcrs_04076', 'vcrs_05621', 'vc..."
5325,ENST00000241453:c.1877_1878insGGGTGACCGGCTCCTC...,"{'vcrs_04650', 'vcrs_04076', 'vcrs_05705', 'vc..."
5388,ENST00000241453:c.1877_1878insCCGTGACCGGCTCCTC...,"{'vcrs_04650', 'vcrs_04076', 'vcrs_05705', 'vc..."
5623,ENST00000241453:c.1815_1816insCCGGCTCCTCAGATAA...,"{'vcrs_04650', 'vcrs_05126', 'vcrs_04076', 'vc..."
5707,ENST00000241453:c.1903_1904insAGTCCACAGGTGACCG...,"{'vcrs_04650', 'vcrs_05126', 'vcrs_04552', 'vc..."


# Imports

In [3]:
import os
import subprocess

import varseek as vk
from varseek.utils import make_vaf_matrix, add_vcf_info_to_cosmic_tsv

# Hyperparameters

In [ ]:
w=54  # window size for varseek build (should be at least 1 less than kallisto k)
k=55
k_standard=31
threads = 32
strand = "unstranded"  # "forward", "reverse", or "unstranded"

# fastq cleanup preceding kb count
trim_edges_off_reads = False
minimum_base_quality_trim_reads=13
qualified_quality_phred = 15
unqualified_percent_limit = 50  # unqualified_percent_limit% of bases in a read must have a quality score of at least qualified_quality_phred
n_base_limit = None
length_required = None

minimum_base_quality_replace_with_N=5
split_reads_by_Ns_and_low_quality_bases = False
run_fastqc = False

# kb count
technology = "bulk"  # same options as kb count
parity = "single"  # single or paired

# vk clean
minimum_count_filter = None  #!!! still need to tune this  
use_binary_matrix = False
drop_zero_columns = False
filter_cells_by_min_counts = True  # True for auto-filter, numeric for set filter, None/False otherwise
filter_cells_by_min_genes = 200
filter_genes_by_min_cells = 3
filter_cells_by_max_mt_content = False
doublet_detection = True
remove_doublets = False
do_cpm_normalization = True
split_reads_by_Ns_and_low_quality_bases = False
dlist_file = None
mutation_metadata_df_columns = None  #!!! replace

# Paths
out_dir_base = "/home/jrich/data/varseek_data_fresh"
reference_out_dir = os.path.join(out_dir_base, "reference")
out_dir_notebook = f"{out_dir_base}/alignment_output"
index_dir = os.path.join(out_dir_base, "vk_build_pipeline_t2t_nov16")
rnaseq_fastq_files = "/home/jrich/data/varseek_data/sequencing/sc/melanoma_10x/10k_Human_DTC_Melanoma_3p_nextgem_fastqs/gex"  # pass as a single string or as a list of strings or a directory to fastq files (if paired-end, then pass each pair sequentially with R1 preceding R2) - if using directory, then it will call order_fastqs_correctly_for_kb_count - don't include index files for single-cell
multiplexed = False  # True if fastq files are multiplexed
seqtk="seqtk"
mutation_index = f"{index_dir}/mutation_reference.idx"
t2g_vk_filter = os.path.join(index_dir, "t2g_filtered.txt")
wt_vcrs_index = mutation_index = f"{index_dir}/wt_vcrs_reference.idx"
wt_vcrs_t2g = os.path.join(out_dir_notebook, "t2g_wt_filtered.txt")
standard_index = ""
standard_t2g = ""
standard_genome = ""  #!!! can use gget ref
standard_gtf = ""  #!!! can use gget ref

In [ ]:
kb_count_out = f"{out_dir_notebook}/kb_count_out"
kb_count_out_wt_vcrs = f"{out_dir_notebook}/kb_count_out_wt_vcrs"
kb_count_out_standard_index = f"{out_dir_notebook}/kb_count_out_standard"
vk_summarize_output_dir = f"{out_dir_notebook}/vk_summarize"
fastqc_out_dir = f"{out_dir_notebook}/fastqc_out"


os.makedirs(out_dir_base, exist_ok=True)
os.makedirs(out_dir_notebook, exist_ok=True)
os.makedirs(reference_out_dir, exist_ok=True)
os.makedirs(kb_count_out, exist_ok=True)
os.makedirs(kb_count_out_wt_vcrs, exist_ok=True)
os.makedirs(kb_count_out_standard_index, exist_ok=True)
os.makedirs(vk_summarize_output_dir, exist_ok=True)
os.makedirs(fastqc_out_dir, exist_ok=True)

assert k >= w + 1, "k must be greater than or equal to w + 1"

if isinstance(rnaseq_fastq_files, str):
    if os.isdir(rnaseq_fastq_files):  # is a directory
        rnaseq_fastq_files = order_fastqs_correctly_for_kb_count(rnaseq_fastq_files, technology=technology, multiplexed=multiplexed)
    else:  # is a space-separated string
        rnaseq_fastq_files = [rnaseq_fastq_files]

adata_path = f"{kb_count_out}/counts_unfiltered/adata.h5ad"
adata_path_wt_vcrs = f"{kb_count_out_wt_vcrs}/counts_unfiltered/adata.h5ad"
adata_path_normal_genome = f"{kb_count_out_standard_index}/counts_unfiltered/adata.h5ad"
mutation_metadata_df_path = os.path.join(index_dir, "variants_updated_filtered.csv")

In [ ]:
rnaseq_fastq_files_list_dict = vk.fastqpp(
    rnaseq_fastq_files_list=rnaseq_fastq_files, 
    trim_edges_off_reads = trim_edges_off_reads, 
    run_fastqc = run_fastqc, 
    split_reads_by_Ns_and_low_quality_bases = split_reads_by_Ns_and_low_quality_bases, 
    parity = parity, 
    technology=technology,
    fastqc_out_dir = fastqc_out_dir,
    minimum_base_quality_trim_reads = minimum_base_quality_trim_reads,
    qualified_quality_phred = qualified_quality_phred,
    unqualified_percent_limit = unqualified_percent_limit,
    n_base_limit = n_base_limit,
    minimum_length = k,
    minimum_base_quality_replace_with_N=minimum_base_quality_replace_with_N,
    fastp = "fastp",
    seqtk = seqtk,
    delete_intermediate_files = False
)

rnaseq_fastq_files_quality_controlled = rnaseq_fastq_files_list_dict["quality_controlled"]
rnaseq_fastq_files_final = rnaseq_fastq_files_list_dict["final"]

# kb count

In [ ]:
if not os.path.exists(kb_count_out) or len(os.listdir(kb_count_out)) == 0:
    kb_count_command = ["kb", "count", "-t", str(threads), "-k", str(k), "-i", mutation_index, "-g", t2g_vk_filter, "-x", technology, "--num", "--h5ad", "--parity", "single", "--strand", strand, "-o", kb_count_out] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

kb count on wt vcrs index

In [ ]:
if os.path.exists(wt_vcrs_index) and (not os.path.exists(kb_count_out_wt_vcrs) or len(os.listdir(kb_count_out_wt_vcrs)) == 0):
    kb_count_command = ["kb", "count", "-t", str(threads), "-k", str(k), "-i", wt_vcrs_index, "-g", wt_vcrs_t2g, "-x", technology, "--num", "--h5ad", "--parity", "single", "--strand", strand, "-o", kb_count_out_wt_vcrs] + rnaseq_fastq_files_final
    subprocess.run(kb_count_command, check=True)

# Optionally, kb ref and count on normal genome

In [ ]:
if not os.path.exists(standard_index) or not os.path.exists(standard_t2g):
    if k_standard == 31:
        kb_ref_command = ["kb", "ref", "-t", str(threads), "-i", standard_index, "-g", standard_t2g, "-d", "human"]
    else:  
        kb_ref_command = ["kb", "ref", "-t", str(threads), "-k", str(k_standard), "-i", standard_index, "-g", standard_t2g, standard_genome, standard_gtf]
    subprocess.run(kb_ref_command, check=True)

# TODO: incorporate technology bulk vs sc in here
if not os.path.exists(kb_count_out_standard_index) or len(os.listdir(kb_count_out_standard_index)) == 0:
    kb_count_standard_index_command = ["kb", "count", "-t", str(threads), "-k", str(k_standard), "-i", standard_index, "-g", standard_t2g, "-x", "bulk", "--h5ad", "--parity", parity, "--strand", strand, "-o", kb_count_out_standard_index] + rnaseq_fastq_files_quality_controlled
    subprocess.run(kb_count_standard_index_command, check=True)

# vk clean

In [ ]:
# TODO: wrap inside a single vk clean command
adata_path_clean = vk.clean(adata_path, output_figures_dir = f"{out_dir_notebook}/vk_clean_figures", mutation_metadata_df = mutation_metadata_df_path, mutation_metadata_df_columns = mutation_metadata_df_columns, minimum_count_filter = minimum_count_filter, use_binary_matrix = use_binary_matrix, drop_zero_columns = drop_zero_columns, filter_cells_by_min_counts = filter_cells_by_min_counts, filter_cells_by_min_genes = filter_cells_by_min_genes, filter_genes_by_min_cells = filter_genes_by_min_cells, filter_cells_by_max_mt_content = filter_cells_by_max_mt_content, doublet_detection = doublet_detection, remove_doublets = remove_doublets, do_cpm_normalization = do_cpm_normalization, adata_path_normal_genome = adata_path_normal_genome, vcrs_id_column = "vcrs_id")

if os.path.exists(adata_path_wt_vcrs):
    adata_path_wt_vcrs_clean = vk.clean(adata_path_wt_vcrs, output_figures_dir = f"{out_dir_notebook}/vk_clean_figures_wt_vcrs", mutation_metadata_df = mutation_metadata_df_path, mutation_metadata_df_columns = mutation_metadata_df_columns, minimum_count_filter = minimum_count_filter, use_binary_matrix = use_binary_matrix, drop_zero_columns = drop_zero_columns, filter_cells_by_min_counts = filter_cells_by_min_counts, filter_cells_by_min_genes = filter_cells_by_min_genes, filter_genes_by_min_cells = filter_genes_by_min_cells, filter_cells_by_max_mt_content = filter_cells_by_max_mt_content, doublet_detection = doublet_detection, remove_doublets = remove_doublets, do_cpm_normalization = do_cpm_normalization, adata_path_normal_genome = adata_path_normal_genome, vcrs_id_column = "vcrs_id")
    
adata_vaf_output_path = make_vaf_matrix(adata_path_clean, adata_path_wt_vcrs_clean)

In [ ]:
import os
import subprocess
import scanpy as sc
import numpy as np
import pandas as pd
import scipy.sparse as sp
import pysam

import varseek as vk
from varseek.utils import make_vaf_matrix, add_vcf_info_to_cosmic_tsv, match_adata_orders, write_to_vcf, write_vcfs_for_rows

adata_path_clean = "/Users/joeyrich/Desktop/local/varseek/data/varseek_count_out_alignment_visualization/kb_count_out_vcrs/counts_unfiltered/adata.h5ad"
cosmic_tsv = "/Users/joeyrich/Desktop/local/varseek/data/reference/cosmic/CancerMutationCensus_AllData_Tsv_v101_GRCh37/CancerMutationCensus_AllData_v101_GRCh37.tsv"
reference_genome_fasta = "/Users/joeyrich/Desktop/local/varseek/data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa"  # grch37, ensembl93
cosmic_cdna_info_csv = "/Users/joeyrich/Desktop/local/varseek/data/reference/cosmic/CancerMutationCensus_AllData_Tsv_v101_GRCh37/CancerMutationCensus_AllData_v101_GRCh37_mutation_workflow.csv"

# I need a metadata df with ID (corresponding to vcrs_header), CHROM, POS, REF, ALT
cosmic_df = add_vcf_info_to_cosmic_tsv(cosmic_tsv=cosmic_tsv, reference_genome_fasta=reference_genome_fasta, cosmic_df_out = None, cosmic_cdna_info_csv = cosmic_cdna_info_csv, mutation_source = "cdna")

adata = sc.read_h5ad(adata_path_clean)

adata.var["vcrs_header"] = adata.var.index  #!!! erase - normally done inside clean
adata.var["vcrs_header_copy"] = adata.var["vcrs_header"]
adata_var_temp = adata.var[['vcrs_header_copy', 'vcrs_header']].copy()

# explode across semicolons
adata_var_temp = adata_var_temp.assign(
    vcrs_header=adata_var_temp["vcrs_header"].str.split(";")
).explode("vcrs_header")

adata_var_temp.reset_index(drop=True, inplace=True)

adata_var_temp = adata_var_temp.merge(cosmic_df, left_on="vcrs_header", right_on="ID", how="left")

In [26]:
adata_var_temp.head()

,vcrs_header,CHROM,POS,ID,REF,ALT
0,ENST00000391429:c.194C>T,17,1174046.0,ENST00000391429:c.194C>T,C,T
1,ENST00000391429:c.128C>A,17,1173980.0,ENST00000391429:c.128C>A,C,A
2,ENST00000391429:c.257C>A,17,1174109.0,ENST00000391429:c.257C>A,C,A
3,ENST00000391429:c.100G>A,17,1173952.0,ENST00000391429:c.100G>A,G,A
4,ENST00000391429:c.99G>A,17,1173951.0,ENST00000391429:c.99G>A,G,A


In [23]:
# grouped_adata_var_temp = (
#     adata_var_temp
#     .groupby("vcrs_header_copy", sort=False)  # Group by vcrs_header_copy while preserving order
#     .agg({
#         "vcrs_header": lambda x: ";".join(x),  # Reconstruct vcrs_header as a single string
#         "CHROM": set,  # Collect CHROM values in the same order as rows
#         "POS": set,    # Collect POS values
#         "REF": set,    # Collect REF values
#         "ALT": set     # Collect ALT values
#     })
#     .reset_index()  # Reset index for cleaner result
# )

for col in ["CHROM", "POS", "REF", "ALT"]:
    grouped_adata_var_temp = grouped_adata_var_temp[grouped_adata_var_temp[col].apply(lambda x: len(set(x)) == 1)]
    grouped_adata_var_temp[col] = grouped_adata_var_temp[col].apply(lambda x: x[0])

adata.var = adata.var.merge(grouped_adata_var_temp, on="vcrs_header", how="left", suffixes=("_original", ""))
adata.var['ID'] = adata.var['ID'].fillna(adata.var['vcrs_header'])

if sp.issparse(adata.X):
    # Sparse matrix handling
    adata.var["NS"] = np.array((adata.X != 0).sum(axis=0)).flatten()
else:
    # Dense matrix handling
    adata.var["NS"] = (adata.X != 0).sum(axis=0)


# adata.var['DP'] = adata.var['vcrs_count'] + adata.var['wt_count']
# adata.var['AF'] = adata.var['vaf_across_samples']
# adata_wt_vcrs = sc.read_h5ad(adata_path_wt_vcrs_clean)
# adata_vaf = sc.read_h5ad(adata_vaf_output_path)

write_to_vcf(adata.var, "output.vcf")


# adata_wt_vcrs = match_adata_orders(adata = adata_wt_vcrs, adata_ref = adata)
# adata_vaf = match_adata_orders(adata = adata_vaf, adata_ref = adata)

# write_vcfs_for_rows(adata = adata, adata_wt_vcrs = adata_wt_vcrs, adata_vaf = adata_vaf, output_dir = ".")

ValueError: cannot perform both aggregation and transformation operations simultaneously

# vk summarize

In [ ]:
vk.summarize(adata_path_clean, technology = technology, output_dir = vk_summarize_output_dir, overwrite = False, top_values = 10)

In [100]:
import pandas as pd

# Define the data
data = {
    "vcrs_header": ["header1", "header2;header3", "header4;header5;header6", "header7", "header8"],
    "vcrs_count": [1, 4, 2, 99, 0],
    "number_obs": [1, 2, 1, 3, 0],
}

# Create the DataFrame
adata_var = pd.DataFrame(data)

In [101]:
adata_var_temp = adata_var.loc[adata_var["vcrs_count"] > 0].copy()
adata_var_temp["vcrs_header_copy"] = adata_var_temp["vcrs_header"]
adata_var_temp.rename(columns={"vcrs_count": "DP", "number_obs": "NS"}, inplace=True)

In [85]:
adata_var_temp

,vcrs_header,DP,NS,vcrs_header_copy
0,header1,1,1,header1
1,header2;header3,4,2,header2;header3
2,header4;header5;header6,2,1,header4;header5;header6
3,header7,99,3,header7


In [102]:
# explode across semicolons
adata_var_temp = adata_var_temp.assign(
    vcrs_header=adata_var_temp["vcrs_header"].str.split(";")
).explode("vcrs_header").reset_index(drop=True)

In [87]:
adata_var_temp

,vcrs_header,DP,NS,vcrs_header_copy
0,header1,1,1,header1
1,header2,4,2,header2;header3
2,header3,4,2,header2;header3
3,header4,2,1,header4;header5;header6
4,header5,2,1,header4;header5;header6
5,header6,2,1,header4;header5;header6
6,header7,99,3,header7


In [103]:
# adata_var_temp = adata_var_temp.merge(cosmic_df, left_on="vcrs_header", right_on="ID", how="left")
adata_var_temp["CHROM"] = ["chr1", "chr2", "chr2", "chr4", "chr4", "chr6", "chr7"]  #!!! temp
adata_var_temp["POS"] = ["1", "2", "2", "4", "4", "6", "7"]  #!!! temp
adata_var_temp["REF"] = ["A", "A", "A", "A", "A", "A", "A"]  #!!! temp
adata_var_temp["ALT"] = ["A", "A", "A", "A", "A", "A", "A"]  #!!! temp

In [104]:
adata_var_temp

,vcrs_header,DP,NS,vcrs_header_copy,CHROM,POS,REF,ALT
0,header1,1,1,header1,chr1,1,A,A
1,header2,4,2,header2;header3,chr2,2,A,A
2,header3,4,2,header2;header3,chr2,2,A,A
3,header4,2,1,header4;header5;header6,chr4,4,A,A
4,header5,2,1,header4;header5;header6,chr4,4,A,A
5,header6,2,1,header4;header5;header6,chr6,6,A,A
6,header7,99,3,header7,chr7,7,A,A


In [110]:
grouped_adata_var_temp = (
    adata_var_temp
    .groupby("vcrs_header_copy", sort=False)  # Group by vcrs_header_copy while preserving order
    .agg({
        "vcrs_header": lambda x: ";".join(x),  # Reconstruct vcrs_header as a single string
        "CHROM": set,  # Collect CHROM values in the same order as rows
        "POS": set,    # Collect POS values
        "REF": set,    # Collect REF values
        "ALT": set,    # Collect ALT values
        "DP": "first",
        "NS": "first",
    })
    .reset_index()  # Reset index for cleaner result
)

grouped_adata_var_temp = grouped_adata_var_temp.rename(columns={"vcrs_header": "ID"}).drop(columns=["vcrs_header_copy"])

In [111]:
grouped_adata_var_temp

,ID,CHROM,POS,REF,ALT,DP,NS
0,header1,{chr1},{1},{A},{A},1,1
1,header2;header3,{chr2},{2},{A},{A},4,2
2,header4;header5;header6,"{chr4, chr6}","{6, 4}",{A},{A},2,1
3,header7,{chr7},{7},{A},{A},99,3


In [112]:
for col in ["CHROM", "POS", "REF", "ALT"]:
    grouped_adata_var_temp = grouped_adata_var_temp[grouped_adata_var_temp[col].apply(lambda x: len(set(x)) == 1)].copy()
    grouped_adata_var_temp[col] = grouped_adata_var_temp[col].apply(lambda x: list(x)[0])

In [113]:
grouped_adata_var_temp

,ID,CHROM,POS,REF,ALT,DP,NS
0,header1,chr1,1,A,A,1,1
1,header2;header3,chr2,2,A,A,4,2
3,header7,chr7,7,A,A,99,3


In [118]:
import importlib
import varseek.utils.varseek_clean_utils
importlib.reload(varseek.utils.varseek_clean_utils)
from varseek.utils.varseek_clean_utils import write_to_vcf
write_to_vcf(grouped_adata_var_temp, "output.vcf")

In [123]:
# make copy column that won't be exploded so that I know how to groupby later
adata_var_temp["vcrs_header_copy"] = adata_var_temp["vcrs_header"]

# rename to have VCF-like column names
adata_var_temp.rename(columns={"vcrs_count": "DP", "number_obs": "NS", "vcrs_header": "ID"}, inplace=True)

# explode across semicolons so that I can merge in vcf_data_df
adata_var_temp = adata_var_temp.assign(
    vcrs_header=adata_var_temp["ID"].str.split(";")
).explode("ID").reset_index(drop=True)

In [124]:
adata_var_temp

,ID,DP,NS,vcrs_header_copy,vcrs_header
0,header1,1,1,header1,[header1]
1,header2;header3,4,2,header2;header3,"[header2, header3]"
2,header4;header5;header6,2,1,header4;header5;header6,"[header4, header5, header6]"
3,header7,99,3,header7,[header7]


In [126]:
adata_var_temp = adata_var.loc[adata_var["vcrs_count"] > 0].copy()  #!!! temp

In [ ]:
def prepare_cleaned_adata_to_become_vcf(adata_var, vcf_data_df, output_vcf = "output.vcf"):
    adata_var_temp = adata_var.loc[adata_var["vcrs_count"] > 0].copy()

    # make copy column that won't be exploded so that I know how to groupby later
    adata_var_temp["vcrs_header_copy"] = adata_var_temp["vcrs_header"]

    # rename to have VCF-like column names
    adata_var_temp.rename(columns={"vcrs_count": "DP", "number_obs": "NS", "vcrs_header": "ID"}, inplace=True)

    # explode across semicolons so that I can merge in vcf_data_df
    adata_var_temp = adata_var_temp.assign(
        ID=adata_var_temp["ID"].str.split(";")
    ).explode("ID").reset_index(drop=True)

    # merge in vcf_data_df (eg cosmic_df)
    if isinstance(vcf_data_df, str) and os.path.isfile(vcf_data_df) and vcf_data_df.endswith(".csv"):
        vcf_data_df = pd.read_csv(vcf_data_df)
    elif isinstance(vcf_data_df, pd.DataFrame):
        pass
    else:
        raise ValueError("vcf_data_df must be a CSV file path or a pandas DataFrame")

    adata_var_temp = adata_var_temp.merge(vcf_data_df, on="ID", how="left")

    # collapse across semicolons so that I get my VCRSs back
    adata_var_temp = (
        adata_var_temp
        .groupby("vcrs_header_copy", sort=False)  # Group by vcrs_header_copy while preserving order
        .agg({
            "ID": lambda x: ";".join(x),  # Reconstruct ID as a single string
            "CHROM": set,  # Collect CHROM values in the same order as rows
            "POS": set,    # Collect POS values
            "REF": set,    # Collect REF values
            "ALT": set,    # Collect ALT values
            "DP": "first",
            "NS": "first",
        })
        .reset_index()  # Reset index for cleaner result
        .drop(columns=["vcrs_header_copy"])
    )

    # only keep the VCRSs that have a single value for CHROM, POS, REF, ALT - there could be some merged headers that have identical VCF information (eg same genomic mutation but for different splice variants), so I can't just drop all merged headers
    for col in ["CHROM", "POS", "REF", "ALT"]:
        adata_var_temp = adata_var_temp[adata_var_temp[col].apply(lambda x: len(set(x)) == 1)].copy()
        adata_var_temp[col] = adata_var_temp[col].apply(lambda x: list(x)[0])

    # write to VCF
    write_to_vcf(adata_var_temp, output_vcf)

# # create vcf_data_df - it needs columns ID (corresponding to vcrs_header from adata.var), CHROM, POS, REF, ALT
# cosmic_df = add_vcf_info_to_cosmic_tsv(cosmic_tsv=cosmic_tsv, reference_genome_fasta=reference_genome_fasta, cosmic_df_out = None, cosmic_cdna_info_csv = cosmic_cdna_info_csv, mutation_source = "cdna")

# prepare_cleaned_adata_to_become_vcf(adata_var=adata.var, vcf_data_df=cosmic_df, output_vcf = "output.vcf")

In [ ]:
import pandas as pd

# Define the data
adata_var_toy = pd.DataFrame({
    "vcrs_header": ["header1", "header2;header3", "header4;header5;header6", "header7", "header8"],
    "vcrs_count": [1, 4, 2, 99, 0],
    "number_obs": [1, 2, 1, 3, 0],
})

cosmic_data_toy = pd.DataFrame({
    "ID": ["header1", "header2", "header3", "header4", "header5", "header6", "header7", "header8"],
    "CHROM": ["chr1", "chr2", "chr2", "chr4", "chr4", "chr6", "chr7", "chr8"],
    "POS": ["1", "2", "2", "4", "4", "6", "7", "8"],
    "REF": ["A", "A", "A", "A", "A", "A", "A", "A"],
    "ALT": ["C", "G", "G", "C", "C", "C", "C", "C"]
})

prepare_cleaned_adata_to_become_vcf(adata_var_toy, vcf_data_df=cosmic_data_toy, output_vcf = "output.vcf")